# First, let's generate a multi-label dataset for the CTBNC classifier

In [3]:
import os
import pandas as pd
import glob
import shutil
from sklearn.metrics import accuracy_score

# Let's use the default CTBNC

https://github.com/dcodecasa/CTBNCToolkit

In [4]:
!java -version

java version "1.8.0_241"
Java(TM) SE Runtime Environment (build 1.8.0_241-b07)
Java HotSpot(TM) 64-Bit Server VM (build 25.241-b07, mixed mode)


In [5]:
os.chdir('..')

In [6]:
!dir

 El volumen de la unidad C no tiene etiqueta.
 El n£mero de serie del volumen es: 9CE1-B000

 Directorio de C:\Users\berna\Downloads\MUIA\TFM\ctbnc-chain\CTBNCToolkit

06/04/2021  22:57    <DIR>          .
06/04/2021  22:57    <DIR>          ..
07/03/2021  20:15    <DIR>          .vscode
04/03/2021  01:10    <DIR>          CTBNCToolkit
06/04/2021  22:57           175.327 CTBNCToolkit.jar
12/04/2021  01:49    <DIR>          data
04/03/2021  01:10    <DIR>          lib
04/03/2021  01:10             1.518 LICENSE
04/03/2021  01:10             5.313 makefile
04/03/2021  01:10               124 MANIFEST.MF
22/04/2021  08:59    <DIR>          notebooks
04/03/2021  01:10             2.126 README.md
12/04/2021  01:48    <DIR>          results
               5 archivos        184.408 bytes
               8 dirs  410.111.782.912 bytes libres


# Define the parameters

In [7]:
inDir = os.path.join('data', 'rehab')
tmpDir = os.path.join('data', 'rehab-tmp')
outDir = os.path.join('results', 'extra')

In [ ]:
features = ['S1','S2','S3','S4','S5','S6','S7','S8','S9','S10','S11','S12',
            'S13','S14','S15','S16','S17','S18','S19','S20','S21','S22','S23',
            'S24','S25','S26','S27','S28','S29']
time = 't'
labels = ['Exercise','ExerciseMode']

# Let's make predictions for each one of the labels independently

In [ ]:
features_cmd = ','.join(f"{item}" for item in features)
features_cmd

In [ ]:
# Train and test separate directories to learn the model.

# !java -jar CTBNCToolkit.jar --v --CTBNC=CTNB --validation=HO,0.75 --validColumns=S1,S2,S3,S4,S5,S6,S7,S8,S9,S10,S11,S12,S13,S14,S15,S16,S17,S18,S19,S20,S21,S22,S23,S24,S25,S26,S27,S28,S29,Exercise --timeName=t --className=ExerciseMode --training=data/rehab-split/train --rPath=results/extra data/rehab-split/test

In [ ]:
# --rPath = {outDir}
for item in labels:
    !java -jar CTBNCToolkit.jar --v --CTBNC=CTNB --validation=HO,0.75 --validColumns={features_cmd} --timeName={time} --className={item} --rPath=results/{item} {inDir}

### Since it works with independent labels, we already have binary relevance implemented. So now let's focus on implementing a chain of classifiers

In [ ]:
inDir = os.path.join('data', 'rehab')
outDir = os.path.join('results', 'extra')

In [ ]:
!java -jar CTBNCToolkit.jar --v --CTBNC=CTNB --validation=HO,0.75 --validColumns=S1,S2,S3,S4,S5,S6,S7,S8,S9,S10,S11,S12,S13,S14,S15,S16,S17,S18,S19,S20,S21,S22,S23,S24,S25,S26,S27,S28,S29,Exercise --timeName=t --className=ExerciseMode --rPath={outDir} {inDir}

# Test how to extract the results from test

In [ ]:
df_metrics = pd.read_csv(os.path.join(outDir, 'metrics.csv'), sep = ';')
df_metrics.head(4)

In [ ]:
#df_methods = pd.read_csv(os.path.join(outDir, 'methodsComparison.csv'), sep = ';')
#df_methods.head(4)

In [8]:
def read_results(res_dir):
    cols = ['id', 'label', 'pred', 'pred_prob']
    valid_cols = [i for i in range(0, 7, 2)] # [0, 2, 4, 6]

    results_file = glob.glob(os.path.join(res_dir, '*-results.txt'))[0]

    df_res = pd.read_csv(results_file, names=cols, skipinitialspace=True, usecols=valid_cols, sep=':|,', engine='python')
    
    return df_res

df = read_results(outDir)
df.head(5)

IndexError: list index out of range

In [ ]:
for row in df.itertuples():
    print(row)
    #tmp_file = os.path.join(tmpDir, file)
    #df_tmp = pd.read_csv(tmp_file)
    #df_tmp.head(3)
    #df_tmp.to_csv(tmp_file, header=True, index=False)

In [ ]:
df_tmp.head(10)

In [9]:
def reset_data(orig_dir, dest_dir):
    try:
        if os.path.exists(dest_dir):
            shutil.rmtree(dest_dir)
        shutil.copytree(orig_dir, dest_dir)
    except:
        return False
    return True

reset_data(inDir, tmpDir)

True

In [ ]:
accuracy_score(df['label'], df['pred'])

# CTBNC chain

In [ ]:
inDir = os.path.join('data', 'rehab')
tmpDir = os.path.join('data', 'rehab-tmp')

features = ['S1','S2','S3','S4','S5','S6','S7','S8','S9','S10','S11','S12',
            'S13','S14','S15','S16','S17','S18','S19','S20','S21','S22','S23',
            'S24','S25','S26','S27','S28','S29']
time = 't'
labels = ['ExerciseMode','Exercise']
models = 'CTNB'
train_perc = 0.75


features_cmd = ','.join(f"{item}" for item in features)
features_cmd

# Restore the tmp directory
reset_data(inDir, tmpDir)

In [10]:
def read_results(res_dir):
    cols = ['id', 'label', 'pred', 'pred_prob']
    valid_cols = [i for i in range(0, 7, 2)] # [0, 2, 4, 6]

    results_file = glob.glob(os.path.join(res_dir, '*-results.txt'))[0]

    df_res = pd.read_csv(results_file, names=cols, skipinitialspace=True, usecols=valid_cols, sep=':|,', engine='python')
    
    return df_res

def reset_data(orig_dir, dest_dir):
    try:
        if os.path.exists(dest_dir):
            shutil.rmtree(dest_dir)
        shutil.copytree(orig_dir, dest_dir)
    except:
        return False
    return True

In [ ]:
for item in labels:
    outDir = os.path.join('results', item) # Generate the directory to store the results
    
    # Run the application to generate, train and test the classifier
    !java -jar CTBNCToolkit.jar --CTBNC={models} --validation=HO,{train_perc} --validColumns={features_cmd} --timeName={time} --className={item} --rPath={outDir} {tmpDir}
    
    # Get the predictions for the test subset
    df_res = read_results(outDir)
    acc = accuracy_score(df_res['label'], df_res['pred'])
    print('Accuracy score for', item, '->', acc )
    
    # Save them for the next model of the chain to use them
    for row in df_res.itertuples():
        file = row[1]
        #label = row[2]
        pred = row[3]
        #prob = row[4]
        tmp_file = os.path.join(tmpDir, file)
        df_tmp = pd.read_csv(tmp_file)
        df_tmp[item] = pred # Set the prediction of the classifier as the input for the next classifier
        df_tmp.to_csv(tmp_file, header=True, index=False)
    
    # Finally append the label to the features for the next model to use it as input
    features.append(item)
    features_cmd = ','.join(f"{item}" for item in features)

In [ ]:
# Restore the tmp directory
reset_data(inDir, tmpDir)

# Binary relevance

In [11]:
inDir = os.path.join('data', 'rehab')
tmpDir = os.path.join('data', 'rehab-tmp')

features = ['S1','S2','S3','S4','S5','S6','S7','S8','S9','S10','S11','S12',
            'S13','S14','S15','S16','S17','S18','S19','S20','S21','S22','S23',
            'S24','S25','S26','S27','S28','S29']
time = 't'
labels = ['ExerciseMode','Exercise']
#models = 'CTBNC3-LL'
models = 'CTNB'
train_perc = 0.75


features_cmd = ','.join(f"{item}" for item in features)
features_cmd

'S1,S2,S3,S4,S5,S6,S7,S8,S9,S10,S11,S12,S13,S14,S15,S16,S17,S18,S19,S20,S21,S22,S23,S24,S25,S26,S27,S28,S29'

In [12]:
for item in labels:
    outDir = os.path.join('results', item) # Generate the directory to store the results
    
    # Run the application to generate, train and test the classifier
    #!java -jar CTBNCToolkit.jar --CTBNC={models} --v --validation=HO,{train_perc} --validColumns={features_cmd} --timeName={time} --className={item} --rPath={outDir} {tmpDir}
    !java -jar CTBNCToolkit.jar --CTBNC={models} --v --validation=CV,10 --validColumns={features_cmd} --timeName={time} --className={item} --rPath={outDir} {tmpDir}
    
    # Get the predictions for the test subset
    df_res = read_results(outDir)
    acc = accuracy_score(df_res['label'], df_res['pred'])
    print('Accuracy score for', item, '->', acc )

Datasets generation:

	. single dataset loading

	. dataset permutation

	. dataset dimension cutting (% = 1.0)

Datasets generation (END)

Learning algorithms generation:

	. model 0: CTNB

		. Mxx_prior = 1.0

		. Tx_prior = 0.005

		. Px_prior = 1.0

		. generation of classification parameter learning algorithm

Learning algorithms generation (END)

Inference algorithm generation:

	. probabilities = true

	. binary class decider = disabled

	. classification algorithm

Inference algorithm generation (END)

Cross validation (10 folds)

Test factories generation:

	. model CTNB

Test factories generation (END)

Execute tests:

	. model M0_CTNB

	. test running



K-Folds Cross Validation schema using 10 folds (leav one out = false).
FOLDED DATASET USED FOR THE 1 TIMES !!!

Cycle 1
Learning: from 1_3_5_5.csv To 4_7_6_5.csv
          from 1_1_1_1.csv To 1_3_5_4.csv excluded
Model learnt in 1.142 seconds

Testing : from 1_1_1_1.csv To 1_3_5_4.csv
Inference on 1_1_1_1.csv
1_1_1_1.csv: Tr

2_7_3_4.csv: True Class: 0, Predicted: 0, Probability: 1.0, Inference time: 0.001
Inference on 2_7_3_5.csv
2_7_3_5.csv: True Class: 0, Predicted: 0, Probability: 0.0, Inference time: 0.001
Inference on 2_7_4_1.csv
2_7_4_1.csv: True Class: 1, Predicted: 0, Probability: 0.9999999999849554, Inference time: 0.002
Inference on 2_7_4_2.csv
2_7_4_2.csv: True Class: 1, Predicted: 0, Probability: 0.999999982905677, Inference time: 0.001
Inference on 2_7_4_3.csv
2_7_4_3.csv: True Class: 1, Predicted: 0, Probability: 0.5591022206955448, Inference time: 0.002
Inference on 2_7_4_4.csv
2_7_4_4.csv: True Class: 1, Predicted: 1, Probability: 0.9999999902424785, Inference time: 0.002
Inference on 2_7_4_5.csv
2_7_4_5.csv: True Class: 1, Predicted: 0, Probability: 0.9369084815175747, Inference time: 0.002
Inference on 2_7_5_1.csv
2_7_5_1.csv: True Class: 1, Predicted: 0, Probability: 0.9999996657943341, Inference time: 0.001
Inference on 2_7_5_2.csv
2_7_5_2.csv: True Class: 1, Predicted: 0, Probability: 

KeyError: 'label'


3_5_5_5.csv: True Class: 1, Predicted: 1, Probability: 1.0, Inference time: 0.002
Inference on 3_5_6_1.csv
3_5_6_1.csv: True Class: 1, Predicted: 1, Probability: 1.0, Inference time: 0.001
Inference on 3_5_6_2.csv
3_5_6_2.csv: True Class: 1, Predicted: 1, Probability: 1.0, Inference time: 0.002
Inference on 3_5_6_3.csv
3_5_6_3.csv: True Class: 1, Predicted: 1, Probability: 1.0, Inference time: 0.001
Inference on 3_5_6_4.csv
3_5_6_4.csv: True Class: 1, Predicted: 1, Probability: 1.0, Inference time: 0.003
Inference on 3_5_6_5.csv
3_5_6_5.csv: True Class: 1, Predicted: 1, Probability: 1.0, Inference time: 0.001
Inference on 3_6_1_1.csv
3_6_1_1.csv: True Class: 0, Predicted: 1, Probability: 1.0, Inference time: 0.003
Inference on 3_6_1_2.csv
3_6_1_2.csv: True Class: 0, Predicted: 0, Probability: 1.0, Inference time: 0.003
Inference on 3_6_1_3.csv
3_6_1_3.csv: True Class: 0, Predicted: 0, Probability: 1.0, Inference time: 0.002
Inference on 3_6_1_4.csv
3_6_1_4.csv: True Class: 0, Predicte

4_4_3_2.csv: True Class: 0, Predicted: 0, Probability: 1.0, Inference time: 0.001
Inference on 4_4_3_3.csv
4_4_3_3.csv: True Class: 0, Predicted: 0, Probability: 1.0, Inference time: 0.002
Inference on 4_4_3_4.csv
4_4_3_4.csv: True Class: 0, Predicted: 0, Probability: 1.0, Inference time: 0.001
Inference on 4_4_3_5.csv
4_4_3_5.csv: True Class: 0, Predicted: 0, Probability: 1.0, Inference time: 0.001
Inference on 4_4_4_1.csv
4_4_4_1.csv: True Class: 1, Predicted: 1, Probability: 0.9841772634017081, Inference time: 0.001
Inference on 4_4_4_2.csv
4_4_4_2.csv: True Class: 1, Predicted: 1, Probability: 0.9992852505987735, Inference time: 0.001
Inference on 4_4_4_3.csv
4_4_4_3.csv: True Class: 1, Predicted: 1, Probability: 0.9998156065186552, Inference time: 0.001
Inference on 4_4_4_4.csv
4_4_4_4.csv: True Class: 1, Predicted: 1, Probability: 0.9999673559306095, Inference time: 0.001
Inference on 4_4_4_5.csv
4_4_4_5.csv: True Class: 1, Predicted: 1, Probability: 0.9962217879160582, Inference

In [ ]:
# Restore the tmp directory
reset_data(inDir, tmpDir)